In [45]:
import pandas as pd

TWEETS_PATH = "congress_labeled.jsonl"

df = pd.read_json(TWEETS_PATH, lines=True)

# convert created_at to datetime
df["created_at_dt"] = pd.to_datetime(df["created_at"], unit="ms", errors="coerce")
df["year"] = df["created_at_dt"].dt.year

print(df[["screen_name", "created_at", "created_at_dt", "year", "stance"]].head())

   screen_name          created_at       created_at_dt  year   stance
0  JohnBoozman 2008-08-04 17:28:51 2008-08-04 17:28:51  2008  Neutral
1  JohnBoozman 2008-08-06 19:04:45 2008-08-06 19:04:45  2008  Neutral
2  JohnBoozman 2008-08-06 20:35:36 2008-08-06 20:35:36  2008  Neutral
3  JohnBoozman 2008-08-07 13:52:52 2008-08-07 13:52:52  2008  Neutral
4  JohnBoozman 2008-08-07 15:12:05 2008-08-07 15:12:05  2008  Neutral


In [46]:
# count tweets per year, stance
counts = df.groupby(["year", "stance"]).size().reset_index(name="n")

# convert to proportions within each year
counts["prop"] = counts.groupby("year")["n"].transform(lambda x: x / x.sum())

# pivot to rows = years, columns = stance
stance_by_year = counts.pivot(index="year", columns="stance", values="prop").fillna(0).round(3)

print(stance_by_year)

stance   Left  Neutral  Right
year                         
2008    0.045    0.804  0.152
2009    0.073    0.736  0.190
2010    0.089    0.669  0.241
2011    0.118    0.634  0.248
2012    0.111    0.641  0.248
2013    0.117    0.606  0.277
2014    0.118    0.612  0.270
2015    0.119    0.591  0.290
2016    0.108    0.565  0.327
2017    0.173    0.483  0.344


In [47]:
import pandas as pd
# load tweets
df_tweets = pd.read_json("congress_labeled.jsonl", lines=True)
df_tweets["created_at_dt"] = pd.to_datetime(df_tweets["created_at"], unit="ms", errors="coerce")
df_tweets["year"] = df_tweets["created_at_dt"].dt.year

# load party labels
party_df = pd.read_parquet("party_labels.parquet")

# merge on screen_name
df_tweets = df_tweets.merge(
    party_df[["screen_name", "party", "office", "state", "district"]],
    on="screen_name",
    how="left",
)

print(df_tweets[["screen_name", "text", "stance", "party", "office", "state", "district"]].head())

   screen_name                                               text   stance  \
0  JohnBoozman  On a conference call about a weekend trip to I...  Neutral   
1  JohnBoozman                          Being interviewed by KTHV  Neutral   
2  JohnBoozman  Being interviewed by KARN on his Arkansas Worl...  Neutral   
3  JohnBoozman              On KWHN in Fort Smith, that's 1320 AM  Neutral   
4  JohnBoozman  Attending a Military of the Purple Heart Cerem...  Neutral   

              party        office     state district  
0  Republican Party  U.S. Senator  Arkansas     None  
1  Republican Party  U.S. Senator  Arkansas     None  
2  Republican Party  U.S. Senator  Arkansas     None  
3  Republican Party  U.S. Senator  Arkansas     None  
4  Republican Party  U.S. Senator  Arkansas     None  


In [48]:
def normalize_party(party: str) -> str:
    p = party.lower()
    if "democrat" in p:
        return "D"
    if "republican" in p or "gop" in p:
        return "R"
    return "O"

df_tweets["party_code"] = df_tweets["party"].apply(normalize_party)

In [49]:
# counts per (party, year, stance)
counts_py = (
    df_tweets
    .groupby(["party_code", "year", "stance"])
    .size()
    .reset_index(name="n")
)

# proportions within each (party, year)
counts_py["prop"] = (
    counts_py
    .groupby(["party_code", "year"])["n"]
    .transform(lambda x: x / x.sum())
)

dem_year = (
    counts_py[counts_py["party_code"] == "D"]
    .pivot(index="year", columns="stance", values="prop")
    .fillna(0)
    .round(3)
)

rep_year = (
    counts_py[counts_py["party_code"] == "R"]
    .pivot(index="year", columns="stance", values="prop")
    .fillna(0)
    .round(3)
)

other_year = (
    counts_py[counts_py["party_code"] == "O"]
    .pivot(index="year", columns="stance", values="prop")
    .fillna(0)
    .round(3)
)

print("Democrats by year:\n", dem_year)
print("Republicans by year:\n", rep_year)
print("Third Party by year:\n", other_year)

Democrats by year:
 stance   Left  Neutral  Right
year                         
2008    0.000    0.583  0.417
2009    0.092    0.714  0.194
2010    0.087    0.720  0.193
2011    0.178    0.619  0.203
2012    0.167    0.647  0.186
2013    0.191    0.576  0.234
2014    0.190    0.571  0.239
2015    0.187    0.554  0.259
2016    0.158    0.533  0.309
2017    0.252    0.407  0.341
Republicans by year:
 stance   Left  Neutral  Right
year                         
2008    0.050    0.830  0.120
2009    0.062    0.749  0.188
2010    0.090    0.647  0.263
2011    0.089    0.643  0.269
2012    0.073    0.637  0.290
2013    0.071    0.624  0.305
2014    0.072    0.638  0.290
2015    0.067    0.619  0.315
2016    0.057    0.598  0.345
2017    0.066    0.584  0.350
Third Party by year:
 stance   Left  Neutral  Right
year                         
2009    0.000    1.000  0.000
2011    0.099    0.462  0.440
2012    0.062    0.625  0.312
2013    0.068    0.685  0.247
2014    0.077    0.674  0.250
2015  

In [50]:
# counts per (state, year, stance)
counts_sy = (
    df_tweets
    .groupby(["state", "year", "stance"])
    .size()
    .reset_index(name="n")
)

counts_sy["prop"] = (
    counts_sy
    .groupby(["state", "year"])["n"]
    .transform(lambda x: x / x.sum())
)

stance_state_year = (
    counts_sy
    .pivot_table(
        index=["state", "year"],
        columns="stance",
        values="prop"
    )
    .fillna(0)
    .reset_index()
)

with pd.option_context("display.max_rows", None):
    print(stance_state_year)

stance           state  year      Left   Neutral     Right
0              Alabama  2009  0.046584  0.866460  0.086957
1              Alabama  2010  0.165877  0.585782  0.248341
2              Alabama  2011  0.110345  0.697701  0.191954
3              Alabama  2012  0.071358  0.710605  0.218038
4              Alabama  2013  0.099127  0.654148  0.246725
5              Alabama  2014  0.102203  0.639648  0.258150
6              Alabama  2015  0.089589  0.597408  0.313003
7              Alabama  2016  0.072929  0.601517  0.325554
8              Alabama  2017  0.105567  0.549755  0.344678
9               Alaska  2009  0.021277  0.702128  0.276596
10              Alaska  2010  0.051095  0.693431  0.255474
11              Alaska  2011  0.067416  0.692884  0.239700
12              Alaska  2012  0.061630  0.793241  0.145129
13              Alaska  2013  0.087054  0.691964  0.220982
14              Alaska  2014  0.051903  0.783737  0.164360
15              Alaska  2015  0.072149  0.677657  0.2501

In [51]:
counts_syo = (
    df_tweets
    .groupby(["state", "year", "office", "stance"])
    .size()
    .reset_index(name="n")
)

counts_syo["prop"] = (
    counts_syo
    .groupby(["state", "year", "office"])["n"]
    .transform(lambda x: x / x.sum())
)

print(counts_syo.head())

     state  year               office   stance    n      prop
0  Alabama  2009             Governor     Left   10  0.088496
1  Alabama  2009             Governor  Neutral   90  0.796460
2  Alabama  2009             Governor    Right   13  0.115044
3  Alabama  2009  U.S. Representative     Left    5  0.023923
4  Alabama  2009  U.S. Representative  Neutral  189  0.904306


In [52]:
map_df = stance_state_year.copy()

# ensure missing stances are 0
for col in ["Left", "Neutral", "Right"]:
    if col not in map_df.columns:
        map_df[col] = 0.0

map_df["prop_right"] = map_df["Right"]
map_df["prop_left"] = map_df["Left"]
map_df["polarization"] = (map_df["Right"] - map_df["Left"]).abs()

In [53]:
state_to_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE",
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR",
    "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
}

map_df["state_abbrev"] = map_df["state"].map(state_to_abbrev)
map_df = map_df[map_df["state_abbrev"].notna()].copy()


In [ ]:
# build per-state/year hover text summarizing office splits
def summarize_office_group(df_group):
    lines = []
    for office, sub in df_group.groupby("office"):
        # pivot stance proportions within this office
        props = (
            sub
            .pivot(index="office", columns="stance", values="prop")
            .fillna(0.0)
        ).iloc[0]
        l = props.get("Left", 0.0)
        n = props.get("Neutral", 0.0)
        r = props.get("Right", 0.0)
        lines.append(f"{office}: L {l:.2f} / N {n:.2f} / R {r:.2f}")
    return "<br>".join(lines)

# prepare counts_syo at same granularity
office_summary = (
    counts_syo
    .groupby(["state", "year"])
    .apply(summarize_office_group)
    .reset_index(name="office_hover_text")
)

# merge into map_df
map_df = map_df.merge(
    office_summary,
    on=["state", "year"],
    how="left"
)

# build overall stance summary for hover
map_df["overall_hover_text"] = (
    "Overall: "
    + "L " + map_df["prop_left"].round(2).astype(str)
    + " / N " + map_df["Neutral"].round(2).astype(str)
    + " / R " + map_df["prop_right"].round(2).astype(str)
)


In [55]:
import plotly.express as px

map_df["year"] = map_df["year"].astype(int)
year_order = sorted(map_df["year"].unique())
map_df["net_lean"] = map_df["prop_left"] - map_df["prop_right"]

fig = px.choropleth(
    map_df,
    locations="state_abbrev",
    locationmode="USA-states",
    color="net_lean",
    scope="usa",
    animation_frame="year",
    category_orders={"year": year_order},
    hover_name="state",
    hover_data={
        "state_abbrev": False,
        "prop_right": ":.2f",
        "prop_left": ":.2f",
        "Neutral": ":.2f",
        "overall_hover_text": True,
        "office_hover_text": False,
    },
    color_continuous_scale=[
        [0.0, "red"],
        [0.5, "white"],
        [1.0, "blue"],
    ],
    range_color=(-1, 1),
)

fig.update_traces(
    hovertemplate=(
        "<b>%{hovertext}</b><br>"
        "Year: %{frame}<br>"
        "Right: %{customdata[0]:.2f}<br>"
        "Neutral: %{customdata[1]:.2f}<br>"
        "Left: %{customdata[2]:.2f}<br>"
        "Leaning: %{customdata[3]:.2f}<br><br>"
        "%{customdata[4]}"
        "<extra></extra>"
    ),
    
    # set customdata manually:
    customdata=map_df[["prop_right", "Neutral", "prop_left", "net_lean","office_hover_text"]].to_numpy()
)

# for f in fig.frames:
#     f.data[0].update(
#     hovertemplate=(
#         "<b>%{hovertext}</b><br>"
#         "Year: %{customdata[0]}<br>"
#         "Right: %{customdata[1]:.2f}<br>"
#         "Neutral: %{customdata[2]:.2f}<br>"
#         "Left: %{customdata[3]:.2f}<br>"
#         "Leaning: %{customdata[4]:.2f}<br><br>"
#         "%{customdata[5]}"
#         "<extra></extra>"
#     ),
    
#     # set customdata manually:
#     customdata=map_df[["year", "prop_right", "Neutral", "prop_left", "net_lean","office_hover_text"]].to_numpy()
# )
    
fig.update_layout(
    title="Predicted stance distribution by state over time",
    coloraxis_colorbar_title="Leaning",
)

fig.show()


In [56]:
# overall label distribution on the Congress tweets
print(df_tweets["stance"].value_counts(normalize=True))

# by year (quick):
print(
    df_tweets
    .groupby("year")["stance"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
    .round(3)
)


stance
Neutral    0.573097
Right      0.302378
Left       0.124524
Name: proportion, dtype: float64
stance   Left  Neutral  Right
year                         
2008    0.045    0.804  0.152
2009    0.073    0.736  0.190
2010    0.089    0.669  0.241
2011    0.118    0.634  0.248
2012    0.111    0.641  0.248
2013    0.117    0.606  0.277
2014    0.118    0.612  0.270
2015    0.119    0.591  0.290
2016    0.108    0.565  0.327
2017    0.173    0.483  0.344


In [57]:
def normalize_party(party: str) -> str:
    if not isinstance(party, str):
        return "Unknown"
    p = party.lower()
    if "democrat" in p:
        return "D"
    if "republican" in p or "gop" in p:
        return "R"
    return "Other"

df_tweets["party_code"] = df_tweets["party"].apply(normalize_party)

df_eval = df_tweets[df_tweets["party_code"].isin(["D", "R"])].copy()

ct = pd.crosstab(df_eval["party_code"], df_eval["stance"], normalize="index")
print(ct)


stance          Left   Neutral     Right
party_code                              
D           0.191848  0.524612  0.283540
R           0.066508  0.614904  0.318588


In [58]:
print(map_df[["state", "year", "prop_left", "prop_right", "Neutral"]].head())

map_df["net_lean"] = map_df["prop_right"] - map_df["prop_left"]
print(map_df["net_lean"].describe())
print("Min net_lean:", map_df["net_lean"].min(), "Max:", map_df["net_lean"].max())

left_states = map_df[map_df["net_lean"] < 0]
print("Num state-years with net_lean < 0:", len(left_states))

# party vs stance confusion matrix
ct = pd.crosstab(df_eval["party_code"], df_eval["stance"], normalize="index")
print(ct)


     state  year  prop_left  prop_right   Neutral
0  Alabama  2009   0.046584    0.086957  0.866460
1  Alabama  2010   0.165877    0.248341  0.585782
2  Alabama  2011   0.110345    0.191954  0.697701
3  Alabama  2012   0.071358    0.218038  0.710605
4  Alabama  2013   0.099127    0.246725  0.654148
count    402.000000
mean       0.158858
std        0.097064
min       -0.166667
25%        0.086227
50%        0.160638
75%        0.232615
max        0.416667
Name: net_lean, dtype: float64
Min net_lean: -0.16666666666666666 Max: 0.4166666666666667
Num state-years with net_lean < 0: 18
stance          Left   Neutral     Right
party_code                              
D           0.191848  0.524612  0.283540
R           0.066508  0.614904  0.318588


In [59]:
per_pol_year = (
    df_tweets
    .groupby(["screen_name", "state", "party_code", "office", "year"])["stance"]
    .value_counts(normalize=True)
    .rename("prop")
    .reset_index()
)

per_pol_year.head()

,screen_name,state,party_code,office,year,stance,prop
0,AkGovBillWalker,Alaska,Other,Governor,2014,Neutral,1.000000
1,AkGovBillWalker,Alaska,Other,Governor,2015,Neutral,0.741617
2,AkGovBillWalker,Alaska,Other,Governor,2015,Right,0.179487
3,AkGovBillWalker,Alaska,Other,Governor,2015,Left,0.078895
4,AkGovBillWalker,Alaska,Other,Governor,2016,Neutral,0.788360


In [60]:
state_year_pol = (
    per_pol_year
    .groupby(["state", "year", "stance"])["prop"]
    .mean()
    .reset_index()
)

# pivot to wide form: one row per state-year
stance_state_year_pol = (
    state_year_pol
    .pivot_table(
        index=["state", "year"],
        columns="stance",
        values="prop",
        fill_value=0.0,
    )
    .reset_index()
)

stance_state_year_pol.head()


stance,state,year,Left,Neutral,Right
0,Alabama,2009,0.052318,0.840875,0.106807
1,Alabama,2010,0.238001,0.573024,0.251967
2,Alabama,2011,0.107180,0.707547,0.185273
3,Alabama,2012,0.074851,0.643571,0.281578
4,Alabama,2013,0.091464,0.617835,0.290701


In [61]:
# ensure columns exist
for col in ["Left", "Neutral", "Right"]:
    if col not in stance_state_year_pol.columns:
        stance_state_year_pol[col] = 0.0

stance_state_year_pol["prop_left"] = stance_state_year_pol["Left"]
stance_state_year_pol["prop_right"] = stance_state_year_pol["Right"]
stance_state_year_pol["neutral_share"] = stance_state_year_pol["Neutral"]

# net lean = Right - Left
stance_state_year_pol["net_lean"] = (
    stance_state_year_pol["prop_right"] - stance_state_year_pol["prop_left"]
)

n_pol = (
    df_tweets
    .groupby(["state", "year"])["screen_name"]
    .nunique()
    .reset_index(name="n_politicians")
)

map_df = stance_state_year_pol.merge(n_pol, on=["state", "year"], how="left")


In [120]:
import plotly.express as px

# per_pol_year: per-politician-year stance distribution
per_pol_year = (
    df_tweets
    .groupby(["screen_name", "state", "party_code", "office", "year"])["stance"]
    .value_counts(normalize=True)
    .rename("prop")
    .reset_index()
)

# aggregate to state × year (each politician weight 1)
state_year_pol = (
    per_pol_year
    .groupby(["state", "year", "stance"])["prop"]
    .mean()
    .reset_index()
)

stance_state_year_pol = (
    state_year_pol
    .pivot_table(
        index=["state", "year"],
        columns="stance",
        values="prop",
        fill_value=0.0,
    )
    .reset_index()
)

# ensure stance columns exist
for col in ["Left", "Neutral", "Right"]:
    if col not in stance_state_year_pol.columns:
        stance_state_year_pol[col] = 0.0

stance_state_year_pol["prop_left"] = stance_state_year_pol["Left"]
stance_state_year_pol["prop_right"] = stance_state_year_pol["Right"]
stance_state_year_pol["neutral_share"] = stance_state_year_pol["Neutral"]

# net lean = Right - Left  (positive = more right)
stance_state_year_pol["net_lean"] = (
    stance_state_year_pol["prop_right"] - stance_state_year_pol["prop_left"]
)

# number of politicians per state-year (for hover)
n_pol = (
    df_tweets
    .groupby(["state", "year"])["screen_name"]
    .nunique()
    .reset_index(name="n_politicians")
)

map_df = stance_state_year_pol.merge(n_pol, on=["state", "year"], how="left")

# state abbreviation
state_to_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE",
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR",
    "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
}

map_df["state_abbrev"] = map_df["state"].map(state_to_abbrev)
map_df = map_df[map_df["state_abbrev"].notna()].copy()

map_df["year"] = map_df["year"].astype(int)
year_order = sorted(map_df["year"].unique())

fig = px.choropleth(
    map_df,
    locations="state_abbrev",
    locationmode="USA-states",
    color="net_lean",
    scope="usa",
    animation_frame="year",
    category_orders={"year": year_order},
    hover_name="state",
    custom_data=[
        "year",
        "prop_left",
        "neutral_share",
        "prop_right",
        "n_politicians",
    ],
    range_color=(-1, 1),
    color_continuous_scale=[
        [0.0, "blue"],   # -1: more left
        [0.5, "white"],  #  0: balanced
        [1.0, "red"],    # +1: more right
    ],
)

fig.update_traces(
    hovertemplate=(
        "<b>%{hovertext}</b><br>"
        "Year: %{customdata[0]}<br>"
        "Left: %{customdata[1]:.2f}<br>"
        "Neutral: %{customdata[2]:.2f}<br>"
        "Right: %{customdata[3]:.2f}<br>"
        "Politicians: %{customdata[4]}<extra></extra>"
    )
)

fig.update_layout(
    title="Per-politician normalized net lean (Right − Left) by state over time",
    coloraxis_colorbar_title="Net lean (Right − Left)",
)

fig.show()


### ERROR ANALYSIS (RIGHT LEANING)

In [63]:
df_pp = df_tweets.copy()

# per-politician, per-year, per-state, per-office stance distribution
per_pol_year = (
    df_pp
    .groupby(["screen_name", "party_code", "state", "office", "year"])["stance"]
    .value_counts(normalize=True)      # proportions within politician-year
    .rename("prop")
    .reset_index()
)

per_pol_year.head()

,screen_name,party_code,state,office,year,stance,prop
0,AkGovBillWalker,Other,Alaska,Governor,2014,Neutral,1.000000
1,AkGovBillWalker,Other,Alaska,Governor,2015,Neutral,0.741617
2,AkGovBillWalker,Other,Alaska,Governor,2015,Right,0.179487
3,AkGovBillWalker,Other,Alaska,Governor,2015,Left,0.078895
4,AkGovBillWalker,Other,Alaska,Governor,2016,Neutral,0.788360


In [64]:
# pivot to one row per (politician, year)
pol_year_wide = (
    per_pol_year
    .pivot_table(
        index=["screen_name", "party_code", "state", "office", "year"],
        columns="stance",
        values="prop",
        fill_value=0.0,
    )
    .reset_index()
)

# ensure stance columns exist
for col in ["Left", "Neutral", "Right"]:
    if col not in pol_year_wide.columns:
        pol_year_wide[col] = 0.0

pol_year_wide["prop_left"] = pol_year_wide["Left"]
pol_year_wide["prop_right"] = pol_year_wide["Right"]
pol_year_wide["prop_neutral"] = pol_year_wide["Neutral"]

# net lean: Right - Left  (positive = right-leaning, negative = left-leaning)
pol_year_wide["net_lean"] = pol_year_wide["prop_right"] - pol_year_wide["prop_left"]

# absolute lean (how strong)
pol_year_wide["abs_net_lean"] = pol_year_wide["net_lean"].abs()

pol_year_wide.head()


stance,screen_name,party_code,state,office,year,Left,Neutral,Right,prop_left,prop_right,prop_neutral,net_lean,abs_net_lean
0,AkGovBillWalker,Other,Alaska,Governor,2014,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
1,AkGovBillWalker,Other,Alaska,Governor,2015,0.078895,0.741617,0.179487,0.078895,0.179487,0.741617,0.100592,0.100592
2,AkGovBillWalker,Other,Alaska,Governor,2016,0.035273,0.788360,0.176367,0.035273,0.176367,0.788360,0.141093,0.141093
3,AkGovBillWalker,Other,Alaska,Governor,2017,0.058824,0.684492,0.256684,0.058824,0.256684,0.684492,0.197861,0.197861
4,AnthonyBrownMD4,D,Maryland,U.S. Representative,2013,0.150568,0.497159,0.352273,0.150568,0.352273,0.497159,0.201705,0.201705


In [66]:
# average per-politician-year leaning by party
print(
    pol_year_wide
    .groupby("party_code")[["prop_left", "prop_neutral", "prop_right", "net_lean"]]
    .mean()
    .round(3)
)

stance      prop_left  prop_neutral  prop_right  net_lean
party_code                                               
D               0.191         0.543       0.266     0.075
Other           0.115         0.639       0.245     0.130
R               0.067         0.620       0.313     0.246


In [67]:
# choose a threshold: e.g. 0.10 = 10 percentage point gap
THRESH = 0.10

# Democrats whose BERT stance is net-right-leaning that year
dem_rightish = pol_year_wide[
    (pol_year_wide["party_code"] == "D") &
    (pol_year_wide["net_lean"] > THRESH)
].sort_values("net_lean", ascending=False)

# Republicans whose BERT stance is net-left-leaning that year
rep_leftish = pol_year_wide[
    (pol_year_wide["party_code"] == "R") &
    (pol_year_wide["net_lean"] < -THRESH)
].sort_values("net_lean", ascending=True)

print("Democrats with right-leaning stance (per-politician-year):")
print(dem_rightish.head(20))

print("\nRepublicans with left-leaning stance (per-politician-year):")
print(rep_leftish.head(20))

Democrats with right-leaning stance (per-politician-year):
stance     screen_name party_code          state               office  year  \
341          JayInslee          D     Washington             Governor  2017   
1592       RepSarbanes          D       Maryland  U.S. Representative  2017   
2489       mikecapuano          D  Massachusetts  U.S. Representative  2013   
1284       RepLipinski          D       Illinois  U.S. Representative  2012   
991        RepGonzalez          D          Texas  U.S. Representative  2017   
772      RepCartwright          D   Pennsylvania  U.S. Representative  2016   
842      RepDannyDavis          D       Illinois  U.S. Representative  2012   
195        EspaillatNY          D       New York  U.S. Representative  2017   
819        RepCummings          D       Maryland  U.S. Representative  2015   
820        RepCummings          D       Maryland  U.S. Representative  2016   
631         RepAlGreen          D          Texas  U.S. Representative  2

In [68]:
pol_year_wide.to_csv("politician_year_leaning_full.csv", index=False)

In [69]:
dem_rightish.to_csv("outlier_dem_rightish.csv", index=False)
rep_leftish.to_csv("outlier_rep_leftish.csv", index=False)

In [112]:
sample = df_tweets[
    (df_tweets["screen_name"] == "RepGonzalez") &
    (df_tweets["year"] == 2017) & (df_tweets["stance"] == "Right")
][["created_at_dt", "stance", "text"]].head(500)

print(sample.to_string())

              created_at_dt stance                                                                                                                                                       text
1215826 2017-05-16 20:35:27  Right                                              RT @McAllenISD: Special US Flag presented to Sanchez Elem. from Congressman Gonzalez! https://t.co/PwQ2eBEEXM
1215827 2017-05-16 20:35:31  Right                                                          RT @McAllenISD: A big welcome from Sanchez Elem. To Congressman Gonzalez! https://t.co/xQAFwq8lbe
1215829 2017-05-16 20:35:35  Right               RT @McAllenISD: Congressman Gonzalez waves to students at Sanchez Elem.! Here celebrating Let's Move/ Active Schools! https://t.co/eCvphI9V…
1215830 2017-05-16 20:35:42  Right                                   RT @McAllenISD: Congressman Gonzalez presents US flag that flew over US capitol to Houston Elem! https://t.co/bjWEbp2Xgi
1215832 2017-05-16 20:35:44  Right                

In [81]:
state_year_pol = (
    pol_year_wide
    .groupby(["state", "year"])[["prop_left", "prop_neutral", "prop_right", "net_lean"]]
    .mean()
    .reset_index()
)

# check California & New York
print(
    state_year_pol[state_year_pol["state"].isin(["California", "New York"])]
    .sort_values(["state", "year"])
    .round(3)
)

stance       state  year  prop_left  prop_neutral  prop_right  net_lean
38      California  2009      0.127         0.760       0.112    -0.015
39      California  2010      0.086         0.767       0.146     0.060
40      California  2011      0.168         0.599       0.233     0.065
41      California  2012      0.145         0.602       0.253     0.108
42      California  2013      0.155         0.586       0.259     0.105
43      California  2014      0.163         0.575       0.262     0.099
44      California  2015      0.166         0.542       0.292     0.126
45      California  2016      0.157         0.493       0.350     0.194
46      California  2017      0.236         0.420       0.343     0.107
254       New York  2009      0.042         0.882       0.076     0.034
255       New York  2010      0.088         0.772       0.141     0.053
256       New York  2011      0.294         0.497       0.210    -0.084
257       New York  2012      0.207         0.569       0.224   

In [82]:
# copy to be safe
df_pp = df_tweets.copy()

# per-politician, per-year, per-state, per-office stance distribution
per_pol_year = (
    df_pp
    .groupby(["screen_name", "party_code", "state", "office", "year"])["stance"]
    .value_counts(normalize=True)
    .rename("prop")
    .reset_index()
)

# pivot to wide: one row per (screen_name, year)
pol_year_wide = (
    per_pol_year
    .pivot_table(
        index=["screen_name", "party_code", "state", "office", "year"],
        columns="stance",
        values="prop",
        fill_value=0.0,
    )
    .reset_index()
)

# ensure stance columns exist
for col in ["Left", "Neutral", "Right"]:
    if col not in pol_year_wide.columns:
        pol_year_wide[col] = 0.0

pol_year_wide["prop_left"] = pol_year_wide["Left"]
pol_year_wide["prop_right"] = pol_year_wide["Right"]
pol_year_wide["prop_neutral"] = pol_year_wide["Neutral"]

# net lean: Right - Left (positive = right-leaning, negative = left-leaning)
pol_year_wide["net_lean"] = pol_year_wide["prop_right"] - pol_year_wide["prop_left"]
pol_year_wide["abs_net_lean"] = pol_year_wide["net_lean"].abs()

# columns we want from the per-politician-year table
per_pol_cols = [
    "screen_name", "party_code", "state", "office", "year",
    "prop_left", "prop_neutral", "prop_right",
    "net_lean", "abs_net_lean",
]

df_with_lean = df_tweets.merge(
    pol_year_wide[per_pol_cols],
    on=["screen_name", "party_code", "state", "office", "year"],
    how="left",
)

df_with_lean[[
    "screen_name", "year", "party_code",
    "stance", "prop_left", "prop_neutral", "prop_right", "net_lean", "text"
]].head()

,screen_name,year,party_code,stance,prop_left,prop_neutral,prop_right,net_lean,text
0,JohnBoozman,2008,R,Neutral,0.0,0.977273,0.022727,0.022727,On a conference call about a weekend trip to I...
1,JohnBoozman,2008,R,Neutral,0.0,0.977273,0.022727,0.022727,Being interviewed by KTHV
2,JohnBoozman,2008,R,Neutral,0.0,0.977273,0.022727,0.022727,Being interviewed by KARN on his Arkansas Worl...
3,JohnBoozman,2008,R,Neutral,0.0,0.977273,0.022727,0.022727,"On KWHN in Fort Smith, that's 1320 AM"
4,JohnBoozman,2008,R,Neutral,0.0,0.977273,0.022727,0.022727,Attending a Military of the Purple Heart Cerem...


In [90]:
THRESH = 0

# Democrats whose per-politician-year lean is right-leaning
dem_rightish_pol = pol_year_wide[
    (pol_year_wide["party_code"] == "D") &
    (pol_year_wide["net_lean"] > THRESH)
].copy()

# Republicans whose per-politician-year lean is left-leaning
rep_leftish_pol = pol_year_wide[
    (pol_year_wide["party_code"] == "R") &
    (pol_year_wide["net_lean"] < -THRESH)
].copy()

print("Dem outlier politician-years:", len(dem_rightish_pol))
print("Rep outlier politician-years:", len(rep_leftish_pol))

Dem outlier politician-years: 813
Rep outlier politician-years: 10


In [91]:
# mark outlier flags on the tweet-level df
df_with_lean["dem_rightish_flag"] = (
    (df_with_lean["party_code"] == "D") &
    (df_with_lean["net_lean"] > THRESH)
)

df_with_lean["rep_leftish_flag"] = (
    (df_with_lean["party_code"] == "R") &
    (df_with_lean["net_lean"] < -THRESH)
)

dem_rightish_tweets = df_with_lean[df_with_lean["dem_rightish_flag"]].copy()
rep_leftish_tweets = df_with_lean[df_with_lean["rep_leftish_flag"]].copy()

print("Tweets from Dem outlier politician-years:", len(dem_rightish_tweets))
print("Tweets from Rep outlier politician-years:", len(rep_leftish_tweets))

Tweets from Dem outlier politician-years: 432664
Tweets from Rep outlier politician-years: 2493


In [92]:
# helper to keep at most N tweets per (screen_name, year)
def head_per_group(df, group_cols, n=100):
    return (
        df
        .sort_values(group_cols + ["created_at_dt"])
        .groupby(group_cols, as_index=False)
        .head(n)
    )

dem_rightish_sample = head_per_group(
    dem_rightish_tweets,
    group_cols=["screen_name", "year"],
    n=100,
)

rep_leftish_sample = head_per_group(
    rep_leftish_tweets,
    group_cols=["screen_name", "year"],
    n=100,
)

In [97]:
pol_year_wide.to_csv("politician_year_leaning_full.csv", index=False)

In [98]:
df_with_lean.to_csv("tweets_with_politician_year_leaning.csv", index=False)

In [96]:
dem_rightish_sample.to_csv("tweets_dem_politicians_right_lean_sampled.csv", index=False)
rep_leftish_sample.to_csv("tweets_rep_politicians_left_lean_sampled.csv", index=False)

In [99]:
dem = df_with_lean[df_with_lean["party_code"] == "D"].copy()

dem_right = dem[dem["stance"] == "Right"].copy()
dem_left  = dem[dem["stance"] == "Left"].copy()
dem_neut  = dem[dem["stance"] == "Neutral"].copy()

print("Dem tweets:", len(dem))
print("Dem→Right:", len(dem_right), "Dem→Left:", len(dem_left), "Dem→Neutral:", len(dem_neut))

Dem tweets: 562633
Dem→Right: 159529 Dem→Left: 107940 Dem→Neutral: 295164


In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np

# Dem tweets predicted Right
err_texts = dem_right["text"].astype(str).tolist()

# Dem tweets predicted Left / Neutral
ref = dem[(dem["stance"].isin(["Left", "Neutral"]))]
ref_texts = ref["text"].astype(str).tolist()

texts = err_texts + ref_texts
y = np.array([1]*len(err_texts) + [0]*len(ref_texts)) # 1 = "Dem labeled as Right", 0 = "Dem labeled as Left/Neutral"

vec = TfidfVectorizer(
    lowercase=True,
    strip_accents="unicode",
    min_df=10,
    ngram_range=(1, 2),
    stop_words="english",
)
X = vec.fit_transform(texts)

clf = LogisticRegression(max_iter=1000)
clf.fit(X, y)

terms = np.array(vec.get_feature_names_out())
coefs = clf.coef_[0]

# Terms most associated with Dem labeled as Right predictions
top_err_idx = np.argsort(-coefs)[:40]
print("Top features pushing Dem tweets into 'Right' bucket:")
for t, w in zip(terms[top_err_idx], coefs[top_err_idx]):
    print(f"{t:30s} {w:.3f}")

# Terms most associated with Dem labeled as Left/Neutral
top_ref_idx = np.argsort(coefs)[:40]
print("\nTop features associated with Dem→(Left/Neutral):")
for t, w in zip(terms[top_ref_idx], coefs[top_ref_idx]):
    print(f"{t:30s} {w:.3f}")

Top features pushing Dem tweets into 'Right' bucket:
disarmhate                     11.978
hillaryclinton                 11.504
realdonaldtrump                11.166
bringbackourgirls              9.592
housedemocrats                 9.570
president trump                8.738
taxpayers                      8.466
taxpayer                       8.202
dems                           8.105
obama                          7.805
repcummings                    7.567
senatorleahy                   7.363
senatorbarb                    7.213
terrorists                     7.177
repsarbanes                    7.084
senatorbaldwin                 6.873
muslimban                      6.792
jerrybrowngov                  6.545
replowenthal                   6.478
senatorcarper                  6.242
governortomwolf                6.221
democrats                      5.937
congressman                    5.912
senatorcardin                  5.875
obamacare                      5.861
betsydevosed       

In [102]:
def add_structure_features(df):
    text = df["text"].astype(str)
    return df.assign(
        is_rt = text.str.startswith("RT "),
        has_mention = text.str.contains("@"),
        has_hashtag = text.str.contains("#"),
        has_url = text.str.contains("http"),
        length = text.str.len(),
    )

dem_right_feat = add_structure_features(dem_right)
ref_feat      = add_structure_features(ref)

def describe_feature(df, name):
    if df[name].dtype == bool:
        return df[name].mean()
    else:
        return df[name].mean()

for feat in ["is_rt", "has_mention", "has_hashtag", "has_url", "length"]:
    print(
        feat,
        "Dem→Right:", describe_feature(dem_right_feat, feat),
        "Dem→(Left/Neutral):", describe_feature(ref_feat, feat),
    )


is_rt Dem→Right: 0.32116417704617967 Dem→(Left/Neutral): 0.18241694451059776
has_mention Dem→Right: 0.664418381610867 Dem→(Left/Neutral): 0.4843960863697706
has_hashtag Dem→Right: 0.5378896626945571 Dem→(Left/Neutral): 0.5326913153925538
has_url Dem→Right: 0.6684552651868939 Dem→(Left/Neutral): 0.6540247678018576
length Dem→Right: 126.2551260272427 Dem→(Left/Neutral): 126.28186522584743


In [115]:
df_with_lean = add_structure_features(df_with_lean)
df_eval = df_with_lean[df_with_lean["party_code"].isin(["D","R"])].copy()

# Confusion matrix by party
print(pd.crosstab(df_eval["party_code"], df_eval["stance"], normalize="index"))

# Also split by RT vs not-RT
for flag, name in [(False, "original"), (True, "retweets")]:
    sub = df_eval[df_eval["is_rt"] == flag]
    print(f"\n{name}:")
    print(pd.crosstab(sub["party_code"], sub["stance"], normalize="index"))


stance          Left   Neutral     Right
party_code                              
D           0.191848  0.524612  0.283540
R           0.066508  0.614904  0.318588

original:
stance          Left   Neutral     Right
party_code                              
D           0.207991  0.544686  0.247323
R           0.070819  0.636274  0.292908

retweets:
stance          Left   Neutral     Right
party_code                              
D           0.135195  0.454163  0.410642
R           0.048575  0.526000  0.425426


In [116]:
dem = df_with_lean[df_with_lean["party_code"] == "D"].copy()
text = dem["text"].str.lower()

dem["mentions_dem"] = text.str.contains("hillary|obama|democrat|democrats|housedemocrats|senatorbaldwin|...")
dem["mentions_gop"] = text.str.contains("gop|republican|republicans|trumpcare|...")

for col in ["mentions_dem", "mentions_gop"]:
    print(f"\n{col}:")
    print(dem.groupby(col)["stance"].value_counts(normalize=True).unstack().round(3))


mentions_dem:
stance         Left  Neutral  Right
mentions_dem                       
False           NaN    0.667  0.333
True          0.192    0.525  0.284

mentions_gop:
stance         Left  Neutral  Right
mentions_gop                       
False           NaN    0.667  0.333
True          0.192    0.525  0.284


In [117]:
party_year = (
    df_eval
    .groupby(["party_code","year"])["stance"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
    .round(3)
)

print(party_year)


stance            Left  Neutral  Right
party_code year                       
D          2008  0.000    0.583  0.417
           2009  0.092    0.714  0.194
           2010  0.087    0.720  0.193
           2011  0.178    0.619  0.203
           2012  0.167    0.647  0.186
           2013  0.191    0.576  0.234
           2014  0.190    0.571  0.239
           2015  0.187    0.554  0.259
           2016  0.158    0.533  0.309
           2017  0.252    0.407  0.341
R          2008  0.050    0.830  0.120
           2009  0.062    0.749  0.188
           2010  0.090    0.647  0.263
           2011  0.089    0.643  0.269
           2012  0.073    0.637  0.290
           2013  0.071    0.624  0.305
           2014  0.072    0.638  0.290
           2015  0.067    0.619  0.315
           2016  0.057    0.598  0.345
           2017  0.066    0.584  0.350


In [121]:
df_tweets = add_structure_features(df_tweets)
orig = df_tweets[~df_tweets["is_rt"]].copy()

# per_pol_year: per-politician-year stance distribution
per_pol_year = (
    orig
    .groupby(["screen_name", "state", "party_code", "office", "year"])["stance"]
    .value_counts(normalize=True)
    .rename("prop")
    .reset_index()
)

# aggregate to state × year (each politician weight 1)
state_year_pol = (
    per_pol_year
    .groupby(["state", "year", "stance"])["prop"]
    .mean()
    .reset_index()
)

stance_state_year_pol = (
    state_year_pol
    .pivot_table(
        index=["state", "year"],
        columns="stance",
        values="prop",
        fill_value=0.0,
    )
    .reset_index()
)

# ensure stance columns exist
for col in ["Left", "Neutral", "Right"]:
    if col not in stance_state_year_pol.columns:
        stance_state_year_pol[col] = 0.0

stance_state_year_pol["prop_left"] = stance_state_year_pol["Left"]
stance_state_year_pol["prop_right"] = stance_state_year_pol["Right"]
stance_state_year_pol["neutral_share"] = stance_state_year_pol["Neutral"]

# net lean = Right - Left  (positive = more right)
stance_state_year_pol["net_lean"] = (
    stance_state_year_pol["prop_right"] - stance_state_year_pol["prop_left"]
)

# number of politicians per state-year (for hover)
n_pol = (
    orig
    .groupby(["state", "year"])["screen_name"]
    .nunique()
    .reset_index(name="n_politicians")
)

map_df = stance_state_year_pol.merge(n_pol, on=["state", "year"], how="left")

# state abbreviation
state_to_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE",
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR",
    "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
}

map_df["state_abbrev"] = map_df["state"].map(state_to_abbrev)
map_df = map_df[map_df["state_abbrev"].notna()].copy()

map_df["year"] = map_df["year"].astype(int)
year_order = sorted(map_df["year"].unique())

fig = px.choropleth(
    map_df,
    locations="state_abbrev",
    locationmode="USA-states",
    color="net_lean",
    scope="usa",
    animation_frame="year",
    category_orders={"year": year_order},
    hover_name="state",
    custom_data=[
        "year",
        "prop_left",
        "neutral_share",
        "prop_right",
        "n_politicians",
    ],
    range_color=(-1, 1),
    color_continuous_scale=[
        [0.0, "blue"],   # -1: more left
        [0.5, "white"],  #  0: balanced
        [1.0, "red"],    # +1: more right
    ],
)

fig.update_traces(
    hovertemplate=(
        "<b>%{hovertext}</b><br>"
        "Year: %{customdata[0]}<br>"
        "Left: %{customdata[1]:.2f}<br>"
        "Neutral: %{customdata[2]:.2f}<br>"
        "Right: %{customdata[3]:.2f}<br>"
        "Politicians: %{customdata[4]}<extra></extra>"
    )
)

fig.update_layout(
    title="Per-politician normalized net lean (Right − Left) by state over time",
    coloraxis_colorbar_title="Net lean (Right − Left)",
)

fig.show()